# DocTable Examples: Update
Here I show how to update data into a DocTable. In addition to providing updated values, DocTable also allows you to create map functions to transform existing data.

In [1]:
import random
import pandas as pd
import numpy as np
import sys
sys.path.append('..')
import doctable

In [2]:
import dataclasses
@dataclasses.dataclass
class Record(doctable.DocTableSchema):
    id: int = doctable.IDCol()
    name: str = doctable.Col(nullable=False)
    age: int = None
    is_old: bool = None
        
def new_db():
    db = doctable.DocTable(schema=Record, target=':memory:', verbose=True)
    N = 10
    for i in range(N):
        age = random.random() # number in [0,1]
        is_old = age > 0.5
        db.insert({'name':'user_'+str(i), 'age':age, 'is_old':is_old}, verbose=False)
    return db

db = new_db()
print(db)

<DocTable (4 cols)::sqlite:///:memory::_documents_>


In [3]:
db.select_df(limit=3)

DocTable: SELECT _documents_.id, _documents_.name, _documents_.age, _documents_.is_old 
FROM _documents_
 LIMIT ? OFFSET ?


,id,name,age,is_old
0,1,user_0,0.493092,False
1,2,user_1,0.400761,False
2,3,user_2,0.483774,False


## Single Update
Update multiple (or single) rows with same values.

In [4]:
db = new_db()
db.select_df(where=db['is_old']==True, limit=3, verbose=False)

,id,name,age,is_old
0,1,user_0,0.604214,True
1,2,user_1,0.545287,True
2,3,user_2,0.785432,True


In [5]:
db = new_db()
db.update({'age':1},where=db['is_old']==True)
db.update({'age':0},where=db['is_old']==False)
db.select_df(limit=3, verbose=False)

DocTable: UPDATE _documents_ SET age=? WHERE _documents_.is_old = 1
DocTable: UPDATE _documents_ SET age=? WHERE _documents_.is_old = 0


,id,name,age,is_old
0,1,user_0,0,False
1,2,user_1,0,False
2,3,user_2,0,False


## Apply as Map Function
This feature allows you to update columns based on the values of old columns.

In [6]:
db = new_db()
values = {db['name']:db['name']+'th', db['age']:db['age']+1, db['is_old']:True}
db.update(values)
db.select_df(limit=3, verbose=False)

DocTable: UPDATE _documents_ SET name=(_documents_.name || ?), age=(_documents_.age + ?), is_old=?


,id,name,age,is_old
0,1,user_0th,1.341306,True
1,2,user_1th,1.306571,True
2,3,user_2th,1.559224,True


## Apply as Set of Ordered Map Functions
This is useful for when the updating of one column might change the value of another, depending on the order in which it was applied.

In [7]:
db = new_db()
values = [(db['name'],db['age']-1), (db['age'],db['age']+1),]
db.update(values)
db.select_df(limit=3, verbose=False)

DocTable: UPDATE _documents_ SET name=(_documents_.age - ?), age=(_documents_.age + ?)


,id,name,age,is_old
0,1,-0.62221288787679,1.377787,False
1,2,-0.989215174598388,1.010785,False
2,3,-0.911748768507688,1.088251,False


## Update Using SQL WHERE String

In [8]:
db = new_db()
db.update({'age':1.00}, wherestr='is_old==true')
db.select_df(limit=5, verbose=False)

DocTable: UPDATE _documents_ SET age=? WHERE is_old==true


,id,name,age,is_old
0,1,user_0,0.347761,False
1,2,user_1,1.000000,True
2,3,user_2,1.000000,True
3,4,user_3,1.000000,True
4,5,user_4,0.104578,False
